In [ ]:
import pandas as pd
import numpy as np
import nltk
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import itertools 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import re

In [ ]:

stop_word_dic={}

with open('sample_data/extra_stop.txt','r') as file:
   
    # reading each line    
    for line in file:
   
        # reading each word        
        for word in line.split():
   
            # displaying the words           
            stop_word_dic[word]=word
with open('sample_data/stop_words_english.txt','r') as file:
   
    # reading each line    
    for line in file:
   
        # reading each word        
        for word in line.split():
   
            # displaying the words           
            stop_word_dic[word]=word



closed_class_stop_words = ['a', 'the', 'an', 'and', 'or', 'but', 'about', 'above', 'after', 'along', 'amid', 'among', \
                           'as', 'at', 'by', 'for', 'from', 'in', 'into', 'like', 'minus', 'near', 'of', 'off', 'on', \
                           'onto', 'out', 'over', 'past', 'per', 'plus', 'since', 'till', 'to', 'under', 'until', 'up', \
                           'via', 'vs', 'with', 'that', 'can', 'cannot', 'could', 'may', 'might', 'must', \
                           'need', 'ought', 'shall', 'should', 'will', 'would', 'have', 'had', 'has', 'having', 'be', \
                           'is', 'am', 'are', 'was', 'were', 'being', 'been', 'get', 'gets', 'got', 'gotten', \
                           'getting', 'seem', 'seeming', 'seems', 'seemed', \
                           'enough', 'both', 'all', 'your' 'those', 'this', 'these', \
                           'their', 'the', 'that', 'some', 'our', 'no', 'neither', 'my', \
                           'its', 'his' 'her', 'every', 'either', 'each', 'any', 'another', \
                           'an', 'a', 'just', 'mere', 'such', 'merely' 'right', 'no', 'not', \
                           'only', 'sheer', 'even', 'especially', 'namely', 'as', 'more', \
                           'most', 'less' 'least', 'so', 'enough', 'too', 'pretty', 'quite', \
                           'rather', 'somewhat', 'sufficiently' 'same', 'different', 'such', \
                           'when', 'why', 'where', 'how', 'what', 'who', 'whom', 'which', \
                           'whether', 'why', 'whose', 'if', 'anybody', 'anyone', 'anyplace', \
                           'anything', 'anytime' 'anywhere', 'everybody', 'everyday', \
                           'everyone', 'everyplace', 'everything' 'everywhere', 'whatever', \
                           'whenever', 'whereever', 'whichever', 'whoever', 'whomever' 'he', \
                           'him', 'his', 'her', 'she', 'it', 'they', 'them', 'its', 'their', 'theirs', \
                           'you', 'your', 'yours', 'me', 'my', 'mine', 'I', 'we', 'us', 'much', 'and/or',"wa","ha",",","n't" ,"”" ,"“" ,"’"
                           ]

In [ ]:

df_true= pd.read_csv("sample_data/true_news.csv")

# New Section

In [ ]:

def make_dic():
  for stop in closed_class_stop_words:
    stop_word_dic[stop]=stop
            
make_dic()  

def clean_me(string):
  return remove_stop_words(word_tokenize(string))

def remove_stop_words(text):
  result=[]
  for word in text:
    word=word.lower()
    word=lemmatizer.lemmatize(word)
    if word in stop_word_dic:
      continue
    else:
      result.append(word)
  return result

In [ ]:
def clean_data_frameTF_IDF(df,col="Statement"):
  text=df[col]
  clean_text=[]
  id=[]
  Link=[]
  df_clean=pd.DataFrame()
  for index, row in df.iterrows():
    text=row[col]
    res=" ".join(clean_me(text))
    clean_text.append(res)
    if col =="Statement":
      id.append(row['Date'])
      Link.append(row['Link'])


  if col =="Statement":
    #, "link":Link
    return pd.DataFrame({'Date': id, 'Clean_text': clean_text}),clean_text
  else:
    return pd.DataFrame({'Clean_tweet': clean_text})

df_clean_true=clean_data_frameTF_IDF(df_true)






In [ ]:
def clean_data_frame(df,AP,col="Statement"):
  text=df[col]
  clean_text=[]
  id=[]
  Link=[]
  df_clean=pd.DataFrame()
  for index, row in df.iterrows():
    text=row[col]
  #text = re.sub(r'@.+', text)
    res=clean_me(text)
    clean_text.append(res)
    if col =="Statement":
      id.append(row['Date'])
      Link.append(row['Link'])
  
  #clean_text.append(clean_me(row[' text']))
  if(AP):
    for index, row in df_pants.iterrows():
      text=row[col]
    #text = re.sub(r'@.+', text)
      res=clean_me(text)
      clean_text.append(res)
      if col =="Statement":
        id.append(row['Date'])
        Link.append(row['Link'])

  if col =="Statement":
    return pd.DataFrame({'Date': id, 'Clean_text': clean_text, "link":Link})
  else:
    return pd.DataFrame({'Clean_tweet': clean_text})

df_clean_true_list=clean_data_frame(df_true,False)


In [ ]:
corpus=df_clean_true[1]
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)


In [ ]:

clustering_true = DBSCAN(eps=0.4, min_samples=6, metric='cosine').fit(X)

In [ ]:
df=df_clean_true[0]
df=pd.concat([df,df["Date"].str.split(expand=True)], axis=1).rename(columns={0:"month",1:"day",2:'year'})
unique, counts = np.unique(clustering_true.labels_, return_counts=True)
counts_cluster=(dict(zip(unique, counts)))
sorted_dict = {}
sorted_keys = sorted(counts_cluster, key=counts_cluster.get, reverse=True) 
for w in sorted_keys:
    sorted_dict[w] = counts_cluster[w]
pd.options.display.max_colwidth = 200
print(sorted_dict)
breaker=0
str_result=[]
d=df
p=False
dic_date={}
for i,x in enumerate(clustering_true.labels_):
  if(x==-1):
    d=df.iloc[[i]]
    if(len(d.loc[d["year"].str.contains(":")])==1):
      continue
    date=int(d["year"])
    Sep = d.loc[(d['Clean_text'].str.contains('year'))]
    if(date in dic_date):
      if((len(Sep["Clean_text"])!=0) or p):
        dic_date[date].append(df_clean_true_list.iloc[i]['Clean_text'])
        if(date==2022):
          if(str_date in date_count):
            date_count[str_date]+=1
          else:
            date_count[str_date]=1
    else:
      if(len(Sep["Clean_text"])!=0 or p):
        dic_date[date]=[(df_clean_true_list.iloc[i]['Clean_text'])]
        if(date==2022):
           if(str_date in date_count):
            date_count[str_date]+=1
           else:
            date_count[str_date]=1
    Sep = d.loc[(d['Clean_text'].str.contains('acc'))]
    if(len(Sep["Clean_text"])!=0):
      pass
    breaker+=1
    
    if(breaker>900):
      break
print("printing")
for k,v in date_count.items():
  if (k[0:3]=="Dec"):
    print(k,v)
   
     
print("printing")
print(str_result)

{-1: 2460}
printing
[]


In [ ]:
for k,v in dic_date.items():
    print (k, len(list(filter(None, v))))

2022 4
2021 5
2020 11
2019 15
2018 15
2017 15
2016 26
2015 24
2014 30
2013 27
2012 50
2011 32
2010 32
2009 7
2008 15
2007 1


In [ ]:
from collections import Counter
bag={}

res=" "
for index in dic_date[2022]:
  
  for el in index:

    if(el in bag):
      bag[el]+=1
    else:
      bag[el]=1
    res=res+" "+str(el)
print(res)
words=res.split(' ')
bigrams = zip(words, words[1:])
bg=[]
for w1 in bigrams:
  bg.append(w1[0]+"_"+w1[1])

counts = Counter(bg)
resy={}
sorted_keys2 = sorted(bag, key=bag.get, reverse=True) 
for w in sorted_keys2:
    resy[w] = bag[w]
sorted_dict1={}
sorted_keys1 = sorted(counts, key=counts.get, reverse=True) 
for w in sorted_keys1:
    sorted_dict1[w] = counts[w]
print(resy)
print(sorted_dict1)

  1 large fully vaccinated teenager 2 large fully vaccinated five- 11- year-olds 17 time hospital vaccinated 20 time die
{'vaccinated': 3, 'large': 2, 'fully': 2, 'time': 2, '1': 1, 'teenager': 1, '2': 1, 'five-': 1, '11-': 1, 'year-olds': 1, '17': 1, 'hospital': 1, '20': 1, 'die': 1}
{'large_fully': 2, 'fully_vaccinated': 2, '_': 1, '_1': 1, '1_large': 1, 'vaccinated_teenager': 1, 'teenager_2': 1, '2_large': 1, 'vaccinated_five-': 1, 'five-_11-': 1, '11-_year-olds': 1, 'year-olds_17': 1, '17_time': 1, 'time_hospital': 1, 'hospital_vaccinated': 1, 'vaccinated_20': 1, '20_time': 1, 'time_die': 1}
